In [29]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
import csv
from dateutil import parser

pd.set_option("display.max_colwidth", None)

In [30]:
def crawl_by_keyword(keyword):
    df=pd.read_csv('thanhnien_'+keyword+'.csv')
    keyword_lookup = keyword
    csv_file = open('thanhnien_'+keyword+'_extracted.csv', 'w', newline='',encoding='utf-8')
    writer = csv.writer(csv_file)
    writer.writerow(['Title', 'URL','Publish Date', 'Mentioned Times','Word Count'])

    for index, row in df.iloc[0:].iterrows():
        title = row['Title']
        url = row['URL']
        news_url = 'https://thanhnien.vn'+url
        response = requests.get(news_url)
        if response.status_code == 403:
            break
        write_flag = True

        soup=BeautifulSoup(response.content,'html.parser')
        try: 
            publish_date = soup.find('div',{'data-role': 'publishdate'}).get_text()
            publish_date=publish_date.replace('\n', ' ').replace('\t', ' ')
            cleaned_publish_date = publish_date.strip()
        except AttributeError:
            print('Could not get date of publication of: \n' + str(news_url))
            write_flag =False

        try:
            news_content = soup.find('div',class_='detail__main').get_text()
            nofm = news_content.lower().count(keyword_lookup.lower())
            nofw = len(news_content.lower().split())
        except  AttributeError:
            print('Cant target news content of: \n' + str(news_url))
            try:
                
                news_content = soup.find('div',class_='detail__cmain').get_text()
                nofm = news_content.lower().count(keyword_lookup.lower())
                nofw = len(news_content.lower().split())
                print('Target cmain instead successfully \n')
            except:
                print('Cant target news content cmain of: \n' + str(news_url) + '\n')
                write_flag = False

        try:
            # Parse the cleaned time string using dateutil.parser
            parsed_time = parser.parse(cleaned_publish_date)
            
            # Extract the date component
            date_only = parsed_time.date()
        except ValueError:
            print("Invalid time format. of: \n" + str(news_url))
        if write_flag == True:
            writer.writerow([title, url,date_only,nofm,nofw])
    csv_file.close()

In [31]:
keywords = ["trí tuệ nhân tạo"]
for text in keywords:
    crawl_by_keyword(text)

Cant target news content of: 
https://thanhnien.vn/co-giam-doc-dieu-hanh-robot-ai-dau-tien-tren-the-gioi-co-duoc-sa-thai-nguoi-185230916123521605.htm
Target cmain instead successfully 

Cant target news content of: 
https://thanhnien.vn/ts-luong-minh-thang-giao-duc-khai-phong-se-tao-ra-dot-pha-trong-linh-vuc-ai-185230912210233818.htm
Target cmain instead successfully 

Could not get date of publication of: 
https://thanhnien.vn/nguoi-tre-tu-cuong-quoc-gia-hung-thinh-185230828154926535.htm
Cant target news content of: 
https://thanhnien.vn/nguoi-tre-tu-cuong-quoc-gia-hung-thinh-185230828154926535.htm
Target cmain instead successfully 

Cant target news content of: 
https://thanhnien.vn/openai-ra-mat-chatgpt-phien-ban-doanh-nghiep-185230830171717381.htm
Target cmain instead successfully 

Cant target news content of: 
https://thanhnien.vn/ai-de-doa-cong-viec-van-phong-185230822152536762.htm
Target cmain instead successfully 

Cant target news content of: 
https://thanhnien.vn/chuyen-dong